In [1]:
import sqlite3
from collections import defaultdict

In [2]:
PATH = '/home/dkbrz/GitHub/lexicography/languages/'

lang1 = 'bg'
lang2 = 'ru'

lang1_token = '{}{}_{}/{}_token.txt'.format(PATH, lang1, lang2, lang1)
lang1_lemma = '{}{}_{}/{}_align.txt'.format(PATH, lang1, lang2, lang1)
lang2_token = '{}{}_{}/{}_token.txt'.format(PATH, lang1, lang2, lang2)
lang2_lemma = '{}{}_{}/{}_align.txt'.format(PATH, lang1, lang2, lang2)

alignment = '{}{}_{}/{}-{}.align'.format(PATH, lang1, lang2, lang1, lang2)
alignment

'/home/dkbrz/GitHub/lexicography/languages/bg_ru/bg-ru.align'

In [3]:
lang1_dict = defaultdict(int)
k = 0
for line in open(lang1_lemma):
    phrase = line.strip().split()
    for word in phrase:
        lang1_dict[word] += 1
        k += 1
k, len(lang1_dict)

(117342707, 1014705)

In [4]:
lang2_dict = defaultdict(int)
k = 0
for line in open(lang2_lemma):
    phrase = line.strip().split()
    for word in phrase:
        lang2_dict[word] += 1
        k += 1
k, len(lang2_dict)

(123086415, 881582)

In [5]:
GOOD = set(['VERB', 'NOUN', 'ADV', 'ADJ'])

In [6]:
n = 0
lang1_set = set()
for i in sorted(lang1_dict, key=lang1_dict.get, reverse=True):
    #print (i, lang1_dict[i])
    data = i.split('_')
    if lang1_dict[i] > 100 and data[1] in GOOD:
        n += 1
        lang1_set.update([i])
        #print (i, lang1_dict[i])
    #if n == 500:
    #    break
n

21649

In [7]:
n = 0
lang2_set = set()
for i in sorted(lang2_dict, key=lang2_dict.get, reverse=True):
    #print (i, lang1_dict[i])
    data = i.split('_')
    if lang2_dict[i] > 100 and data[1] in GOOD:
        n += 1
        lang2_set.update([i])
        #print (i, lang1_dict[i])
    #if n == 500:
    #    break
n

22730

In [8]:
len(lang1_set), len(lang2_set)

(21649, 22730)

In [9]:
files = [open(lang1_token), open(lang1_lemma), open(lang2_token), open(lang2_lemma), open(alignment)]

for lines in zip(*files):
    #assert len(lines[0].split()) == len(lines[1].split()) 
    print (lines)
    align_phrase = [tuple(int(x) for x in i.split('-')) for i in lines[-1].split()]
    print (align_phrase)
    lang1_phrase = lines[1].split()
    lang2_phrase = lines[3].split()
    for i in align_phrase:
        print (lang1_phrase[i[0]], lang2_phrase[i[1]])
    break

('Досега в " Сибир " ... Тигър !\n', 'досега_ADV_0 в_ADP_0 "_PUNCT_0 Сибир_PROPN_0 "_PUNCT_0 ..._PUNCT_0 тигър_NOUN_0 !_PUNCT_0\n', 'Ранее в Сибири . . . ( Неразборчиво кричит )\n', 'ранее_ADV_0 в_ADP_1 Сибирь_PROPN_0 ._PUNCT_0 ._PUNCT_0 ._PUNCT_0 (_PUNCT_0 неразборчиво_ADV_0 кричать_VERB_0 )_PUNCT_0\n', '0-0 1-1 3-2 4-6 5-3 5-4 5-5 6-7 7-9\n')
[(0, 0), (1, 1), (3, 2), (4, 6), (5, 3), (5, 4), (5, 5), (6, 7), (7, 9)]
досега_ADV_0 ранее_ADV_0
в_ADP_0 в_ADP_1
Сибир_PROPN_0 Сибирь_PROPN_0
"_PUNCT_0 (_PUNCT_0
..._PUNCT_0 ._PUNCT_0
..._PUNCT_0 ._PUNCT_0
..._PUNCT_0 ._PUNCT_0
тигър_NOUN_0 неразборчиво_ADV_0
!_PUNCT_0 )_PUNCT_0


In [10]:
files = [open(lang1_token), open(lang1_lemma), open(lang2_token), open(lang2_lemma), open(alignment)]

over100_both = 0
under100_both = 0
diff = 0
for lines in zip(*files):
    #assert len(lines[0].split()) == len(lines[1].split()) 
    #print (lines)
    align_phrase = [tuple(int(x) for x in i.split('-')) for i in lines[-1].split()]
    #print (align_phrase)
    lang1_phrase = lines[1].split()
    lang2_phrase = lines[3].split()
    for i in align_phrase:
        #print (lang1_phrase[i[0]], lang2_phrase[i[1]])
        if lang1_phrase[i[0]] in lang1_set and lang2_phrase[i[1]] in lang2_set:
            over100_both += 1
        elif lang1_phrase[i[0]] not in lang1_set and lang2_phrase[i[1]] not in lang2_set:
            under100_both += 1
        else:
            diff += 1
over100_both, under100_both, diff
#(5508537, 513065, 1371246)

(33228932, 67018073, 30407034)

In [11]:
s = over100_both + under100_both +diff
over100_both/s, under100_both/s, diff/s

(0.25432762932036107, 0.5129429867835926, 0.23272938389604625)

In [12]:
import warnings
warnings.filterwarnings("ignore")
from collections import defaultdict
from tqdm.autonotebook import tqdm

import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd='password',
  database="test_bilingual")
mycursor = mydb.cursor()

In [13]:
lang = 4
for i in sorted(lang1_set):
    data = i.split('_')
    sense = int(data[-1])
    pos = data[-2]
    lemma = '_'.join(data[:-2])
    #print (lemma, pos, data)
    if pos == 'PUNCT':
        pass
    else:
        mycursor.execute('INSERT INTO dictionary (lang, lemma, pos, sense) VALUES (%s, %s, %s, %s)',
                        (lang, lemma, pos, sense))

In [14]:
mycursor.execute('SELECT * FROM dictionary WHERE lang=2')
result = mycursor.fetchall()
lang_dict = {}
for i in result:
    lang_dict['{}_{}_{}'.format(i[2], i[3], i[4])] = i[0]
del result

In [16]:
lang = 2
l = 0
for i in sorted(lang2_set):
    if i not in lang_dict:
        data = i.split('_')
        sense = int(data[-1])
        pos = data[-2]
        lemma = '_'.join(data[:-2])
        #print (lemma, pos, data)
        if pos == 'PUNCT':
            pass
        else:
            mycursor.execute('INSERT INTO dictionary (lang, lemma, pos, sense) VALUES (%s, %s, %s, %s)',
                            (lang, lemma, pos, sense))
            #pass
        l += 1
l

21

In [17]:
mydb.commit()

In [18]:
del lang1_set, lang2_set
del lang1_dict, lang2_dict

In [19]:
mycursor.execute('SELECT * FROM dictionary WHERE lang=4')
result = mycursor.fetchall()
lang1_dict = {}
for i in result:
    lang1_dict['{}_{}_{}'.format(i[2], i[3], i[4])] = i[0]
del result

In [20]:
mycursor.execute('SELECT * FROM dictionary WHERE lang=2')
result = mycursor.fetchall()
lang2_dict = {}
for i in result:
    lang2_dict['{}_{}_{}'.format(i[2], i[3], i[4])] = i[0]
del result

In [21]:
len(lang1_dict), len(lang2_dict)

(21649, 27514)

In [6]:
lang1_dict['do_VERB_0']

733

In [22]:
mycursor.execute('SELECT * FROM examples ORDER BY id DESC LIMIT 1')
mycursor.fetchall()

[(15367602,
  'Переклад : Байда До зустрічі в новому сезоні !',
  'Переведено специально для aMovies.biz',
  1,
  2)]

In [23]:
def separate_word(item):
    data = i.split('_')
    sense = int(data[-1])
    pos = data[-2]
    lemma = '_'.join(data[:-2])
    return lemma, pos, sense

In [23]:
mydb.rollback()

In [24]:
files = [open(lang1_token), open(lang1_lemma), open(lang2_token), open(lang2_lemma), open(alignment)]

k = 15367603
for lines in tqdm(zip(*files)):
    left_token = lines[0].strip()
    right_token = lines[2].strip()
    lang1_phrase = lines[1].split()
    lang2_phrase = lines[3].split()
    align_phrase = [tuple(int(x) for x in i.split('-')) for i in lines[-1].split()]
    
    tot = 0
    for i in align_phrase:
        left = lang1_phrase[i[0]]
        right = lang2_phrase[i[1]]
        if left in lang1_dict and right in lang2_dict:
            mycursor.execute("INSERT INTO alignment (id_sent, id_left, id_right, id_lemma_left, id_lemma_right) \
             VALUES (%s, %s, %s, %s, %s)", (k, i[0], i[1], lang1_dict[left], lang2_dict[right],))
            tot += 1
    
    if tot > 0:
        mycursor.execute("INSERT INTO examples (sent_left, sent_right, lang_left, lang_right) VALUES (%s, %s, 1, 2)",
                    (left_token, right_token,))
        k += 1

DataError: 1264 (22003): Out of range value for column 'id_right' at row 1

In [25]:
mydb.commit()

In [13]:
mycursor.execute('SELECT DISTINCT id_sent FROM alignment')
set_ids = set(i[0] for i in mycursor.fetchall())

In [14]:
len(set_ids)

594751

In [15]:
mycursor.execute('SELECT DISTINCT id FROM examples')
set_ids = set(i[0] for i in mycursor.fetchall())

In [16]:
len(set_ids)

594751